In [ ]:
import os
import json
import time
import gzip
import glob
import pickle
import datetime
import numpy as np
import pandas as pd
# from scipy import stats
import random
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("ticks")

def timeStamped(fname, fmt='%Y-%m-%d-%H-%M-%S_{fname}'):
    return datetime.datetime.now().strftime(fmt).format(fname=fname)

def simulate(all_comb_, df_res_sum, df_venue_sum):
    rv_list = []
    for i, j in enumerate(all_comb_):
        if i%1000 ==0:
            print(i)
        rv = []
        b = df_res_sum.budget[i]
        lowest_cost = j.cost.min()
        sampled_sort = np.random.choice(j.index, p=j.prob_rv, size=len(j), replace=False)
        for choice_index in sampled_sort:
            if j.cost[choice_index] <= b:
                rv.append([df_res_sum.id[i], df_venue_sum.id[choice_index]])
                b = b - j.cost[choice_index]
            if b < lowest_cost:
                break
        rv_list.append(rv) 
    
    return rv_list

sample_res = pd.read_pickle('./res_summaries/2019-04-04-20-47-06_sample_res.pkl.gz')
sample_venue = pd.read_pickle('./venue_summaries/2019-04-04-20-47-06_sample_venue.pkl.gz')
sample_comb = pd.read_pickle('./cost_summaries/2019-04-04-20-47-06_cost_sample_pairs.pkl')

stdevs = np.arange(0.1, 0.6, 0.1)
stdevs = np.around(stdevs, 2)
budget_means = np.arange(0.01,4.06,0.05)

std_done = []
for std in stdevs:
    if std not in std_done:
        print('STD: ', std)
        mu_done = []
        # np.random.seed(0) # REMOVE
        for mu in budget_means:
            if mu not in mu_done:
                print('BUDGET MEAN: ', mu)
        #         sample_res['budget'] = mu * np.ones(sample_res.shape[0])
                sample_res['budget'] = np.random.normal(mu, std, sample_res.shape[0])
                fname2 = timeStamped('_sample_res_%s_%s.pkl') % (mu, std)
                sample_res.to_pickle('./res_summaries/varying_mean/%s/trial_std/' % len(sample_res) + fname2)
                print('budget stats: ')
                print(sample_res.budget.min(), ' , ' , sample_res.budget.median(), ' , ' , sample_res.budget.max())
                print('researchers with negative beginning budget: ', sample_res.loc[sample_res.budget <= 0].shape)

                rv_pairs = simulate(sample_comb, sample_res, sample_venue)

                fname3 = timeStamped('rv_pairs_%s_%s.pkl.gz') % (mu, std)
                fp = gzip.open(os.path.join('./rv_combinations/varying_mean/%s/trial_std/' % len(sample_res), fname3), 'wb')
                pickle.dump(rv_pairs, fp)
                fp.close()

                mu_done.append(mu)
        std_done.append(std)